## Energy saved from recycling
<p>Did you know that recycling saves energy by reducing or eliminating the need to make materials from scratch? For example, aluminum can manufacturers can skip the energy-costly process of producing aluminum from ore by cleaning and melting recycled cans. Aluminum is classified as a non-ferrous metal.</p>
<p>Singapore has an ambitious goal of becoming a zero-waste nation. The amount of waste disposed of in Singapore has increased seven-fold over the last 40 years. At this rate, Semakau Landfill, Singapore’s only landfill, will run out of space by 2035. Making matters worse, Singapore has limited land for building new incineration plants or landfills.</p>
<p>The government would like to motivate citizens by sharing the total energy that the combined recycling efforts have saved every year. They have asked you to help them.</p>
<p>You have been provided with three datasets. The data come from different teams, so the names of waste types may differ.</p>
<div style="background-color: #efebe4; color: #05192d; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6;">
    <div style="font-size:16px"><b>datasets/wastestats.csv - Recycling statistics per waste type for the period 2003 to 2017</b>
    </div>
    <div>Source: <a href="https://www.nea.gov.sg/our-services/waste-management/waste-statistics-and-overall-recycling">Singapore National Environment Agency</a></div>
<ul>
    <li><b>waste_type: </b>The type of waste recycled.</li>
    <li><b>waste_disposed_of_tonne: </b>The amount of waste that could not be recycled (in metric tonnes).</li>
    <li><b>total_waste_recycle_tonne: </b>The amount of waste that could be recycled (in metric tonnes).</li>
    <li><b>total_waste_generated: </b>The total amount of waste collected before recycling (in metric tonnes).</li>
    <li><b>recycling_rate: </b>The amount of waste recycled per tonne of waste generated.</li>
    <li><b>year: </b>The recycling year.</li>
</ul>
    </div>
<div style="background-color: #efebe4; color: #05192d; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6; margin-top: 17px;">
    <div style="font-size:16px"><b>datasets/2018_2019_waste.csv - Recycling statistics per waste type for the period 2018 to 2019</b>
    </div>
    <div> Source: <a href="https://www.nea.gov.sg/our-services/waste-management/waste-statistics-and-overall-recycling">Singapore National Environment Agency</a></div>
<ul>
    <li><b>Waste Type: </b>The type of waste recycled.</li>
    <li><b>Total Generated: </b>The total amount of waste collected before recycling (in thousands of metric tonnes).</li> 
    <li><b>Total Recycled: </b>The amount of waste that could be recycled. (in thousands of metric tonnes).</li>
    <li><b>Year: </b>The recycling year.</li>
</ul>
    </div>
<div style="background-color: #efebe4; color: #05192d; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6; margin-top: 17px;">
    <div style="font-size:16px"><b>datasets/energy_saved.csv -  Estimations of the amount of energy saved per waste type in kWh</b>
    </div>
<ul>
    <li><b>material: </b>The type of waste recycled.</li>
    <li><b>energy_saved: </b>An estimate of the energy saved (in kiloWatt hour) by recycling a metric tonne of waste.</li> 
    <li><b>crude_oil_saved: </b>An estimate of the number of barrels of oil saved by recycling a metric tonne of waste.</li>

In [120]:
import pandas as pd
waste03 = pd.read_csv("datasets/wastestats.csv")
waste18 = pd.read_csv("datasets/2018_2019_waste.csv")

In [132]:
waste_types_lower = set(
    waste03.waste_type.str.lower().unique().tolist() + 
    waste18["Waste Type"].str.lower().unique().tolist())

In [122]:
valid_waste = {'ferrous metal',
    'ferrous metals', 
    'non-ferrous metal',
    'non-ferrous metals', 
    'plastic',
    'plastics',
    'glass'}

In [123]:
#confirm our valid_waste variable didn't miss any waste types
waste_types_lower.difference(valid_waste)

{'ash & sludge',
 'ash and sludge',
 'c&d',
 'construction debris',
 'construction& demolition',
 'food',
 'food waste',
 'horticultural',
 'horticultural waste',
 'others',
 'others (stones, ceramic, rubber, ect)',
 'others (stones, ceramic, rubber, etc.)',
 'others (stones, ceramics & rubber etc)',
 'others (stones, ceramics & rubber etc.)',
 'overall',
 'paper/cardboard',
 'scrap tyres',
 'sludge',
 'textile/leather',
 'total',
 'used slag',
 'wood',
 'wood/timber'}

In [124]:
waste03["Total Generated ('000 tonnes)"] = waste03["total_waste_generated_tonne"]/1000
waste03["Total Recycled ('000 tonnes)"] = waste03["total_waste_recycled_tonne"]/1000

unused_columns = ["recycling_rate", "total_waste_generated_tonne", "total_waste_recycled_tonne", "waste_disposed_of_tonne"]

agg_df = waste03.drop(unused_columns, axis=1)\
    .rename({"waste_type":"Waste Type", "year":"Year"}, axis=1)\
    .append(waste18, ignore_index=True)

In [125]:
with open(r"datasets/energy_saved.csv", "r") as file:
    print(file.read().lower())

the table gives the amount of energy saved in kilowatt hour (kwh) and the amount of crude oil (barrels) by recycling 1 metric tonne (1000 kilogram)  per waste type,,,,,
1 barrel oil is approximately 159 litres of oil,,,,,
,,,,,
material,plastic,glass,ferrous metal,non-ferrous metal,paper
energy_saved,5774 kwh,42 kwh,642 kwh,14000 kwh,4000 kwh
crude_oil saved,16 barrels,,1.8 barrels,40 barrels,1.7 barrels


The table gives the amount of energy saved in kilowatt hour (kWh) and the amount of crude oil (barrels) by recycling 1 metric tonne (1000 kilogram) per waste type

In [126]:
energy_saved = pd.read_csv("datasets/energy_saved.csv", header=3, index_col=0)
energy_saved.columns = energy_saved.columns.str.lower()
energy_saved.T

material,energy_saved,crude_oil saved
plastic,5774 Kwh,16 barrels
glass,42 Kwh,NaN
ferrous metal,642 Kwh,1.8 barrels
non-ferrous metal,14000 Kwh,40 barrels
paper,4000 kWh,1.7 barrels


In [128]:
cleaned_df = agg_df[agg_df["Waste Type"].str.lower().isin(valid_waste)]
cleaned_df["Waste Type"] = cleaned_df["Waste Type"].str.lower()
cleaned_df["Waste Type"].replace(
    {"non-ferrous metals": "non-ferrous metal",
     "ferrous metals": "ferrous metal",
     "plastics": "plastic"}, inplace=True)

In [129]:
cleaned_df = cleaned_df.merge(energy_saved.T, left_on = "Waste Type", right_index=True, how="left")
cleaned_df["energy_saved_int"] = cleaned_df.energy_saved.apply(lambda x:x.strip(" Kwh")).astype("int64")
cleaned_df["total_energy_saved"] = cleaned_df["energy_saved_int"] * cleaned_df["Total Recycled ('000 tonnes)"]

In [130]:
annual_energy_savings = cleaned_df[cleaned_df["Year"].isin([2015,2016,2017,2018,2019])].pivot_table(index="Year", values="total_energy_saved", aggfunc="sum")
annual_energy_savings["total_energy_saved"] = annual_energy_savings["total_energy_saved"] * 1000
annual_energy_savings.index.rename("year", inplace=True)